### Making Data Ready

In [1]:
import random
import copy 
import pandas as pd # type: ignore
import numpy as np # type: ignore
import geopandas as gpd # type: ignore
import folium # type: ignore

from shapely.wkt import loads # type: ignore
from math import  sin, cos, sqrt, atan2, radians
from shapely.geometry import Point, LineString # type: ignore

random.seed(100)


In [2]:
ontario_map = "../data/maps/ontario.geojson"
file_path_highway = "../data/raw/shapefile_data.csv"
file_path_gas = "../data/raw/ontario_gas_stations.csv"
file_path_pop = "../data/raw/canada_data.xlsx"
file_path_malls = "../data/raw/malls.csv"
file_path_parking = "../data/maps/parking.kml"

ontario_map_gdf = gpd.read_file(ontario_map).to_crs(epsg=3857)
dummy_map = ontario_map_gdf.explode(ignore_index=True).iloc[[2]]
dummy_map['Boundary'] = 'Prince-Edward-County'

# Load highway data
df_highway = pd.read_csv(file_path_highway)
df_highway["geometry"] = df_highway["geometry"].apply(loads)
gdf_full = gpd.GeoDataFrame(df_highway, geometry="geometry")

# Load gas station data
df_gas_station = pd.read_csv(file_path_gas)

# Load population data
df_pop = pd.read_excel(file_path_pop, engine="openpyxl")
df_pop['geometry'] = df_pop.apply(lambda row: Point(row['long'], row['lat']), axis=1)
gdf_pop = gpd.GeoDataFrame(df_pop, geometry="geometry")

# Load mall data
df_mall = pd.read_csv(file_path_malls)
df_mall['geometry'] = df_mall.apply(lambda row: Point(row['long'], row['lat']), axis=1)
gdf_mall = gpd.GeoDataFrame(df_mall, geometry="geometry", crs="EPSG:4326")

# Load parking data from KML file
gdf_parking = gpd.read_file(file_path_parking, driver='KML')

In [3]:
# Filter for Prince Edward County
gdf = gdf_full[(gdf_full["CSDNAME_L"] == "Prince Edward County") | (gdf_full["CSDNAME_R"] == "Prince Edward County")]
gdf = gdf.set_crs(epsg=3347, allow_override=True).to_crs(epsg=4326)

In [4]:
# Define bounding box for Prince Edward County
prince_edward_bounds = {
    "min_lat": 43.85,
    "max_lat": 44.10,
    "min_lon": -77.65,
    "max_lon": -76.90
}

# Filter gas stations within the bounding box
df_prince_edward = df_gas_station[
    (df_gas_station["Latitude"] >= prince_edward_bounds["min_lat"]) &
    (df_gas_station["Latitude"] <= prince_edward_bounds["max_lat"]) &
    (df_gas_station["Longitude"] >= prince_edward_bounds["min_lon"]) &
    (df_gas_station["Longitude"] <= prince_edward_bounds["max_lon"])
]

In [5]:
def monte_carlo_sampling(polygon, num_points=10):
    """
    Generates a specified number of random points within a given polygon.

    This function samples random points by generating coordinates within the 
    bounding box of the polygon and checking if they fall inside the polygon. 
    It continues generating points until the required number is reached.

    Parameters:
    - polygon (shapely.geometry.Polygon): The polygon within which points should be sampled.
    - num_points (int, optional): The number of random points to generate. Default is 100.

    Returns:
    - list of shapely.geometry.Point: A list of points inside the polygon.

    Note:
    - This method uses a rejection sampling approach, which may be inefficient 
      for complex or irregularly shaped polygons with large holes.
    """
    points = []
    minx, miny, maxx, maxy = polygon.bounds
    while len(points) < num_points:
        random_point = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if polygon.contains(random_point):
            points.append(random_point)
    return points

In [6]:
def plot_sampled_points(sampled_points, base_map, map_object):
    """
    Plots sampled points on a Folium map as a separate layer.

    This function takes a list of sampled points, converts them into a GeoDataFrame, 
    ensures they have the correct CRS (EPSG:4326) for mapping, and adds them to 
    the provided Folium map as a `FeatureGroup`.

    Parameters:
    - sampled_points (list of shapely.geometry.Point): List of sampled points to be plotted.
    - base_map (geopandas.GeoDataFrame): The base map containing spatial reference (CRS).
    - map_object (folium.Map): The Folium map to which the points will be added.

    Returns:
    - folium.Map: The updated map with the sampled points layer.

    Note:
    - The function automatically reprojects the points and the base map to EPSG:4326 if needed.
    - The added layer can be toggled using the LayerControl feature.
    """

    # Convert points to GeoDataFrame
    points_gdf = gpd.GeoDataFrame(geometry=sampled_points, crs=base_map.crs)

    # Ensure CRS is EPSG:4326 for mapping
    if points_gdf.crs.to_string() != "EPSG:4326":
        points_gdf = points_gdf.to_crs(epsg=4326)
    if base_map.crs.to_string() != "EPSG:4326":
        base_map = base_map.to_crs(epsg=4326)

    # Create FeatureGroup for points
    point_layer = folium.FeatureGroup(name="Sampled Points", overlay=True, control=True)

    # Add each point as a CircleMarker
    for _, row in points_gdf.iterrows():
        point = row.geometry
        folium.CircleMarker(
            location=[point.y, point.x],  # (lat, lon)
            radius=3,
            color='red',
            fill=True,
            fill_color='red',
            fill_opacity=0.7
        ).add_to(point_layer)

    # Add the points layer to the map
    point_layer.add_to(map_object)

    # Ensure LayerControl is added for toggling layers
    # folium.LayerControl(collapsed=False).add_to(map_object)

    return map_object,point_layer,points_gdf  # Return the updated map

In [7]:
m = folium.Map(location=[44, -77.2], zoom_start=11)

boundary_layer = folium.FeatureGroup(name="Prince-Edward-County Boundary", 
                                     overlay=True, control=True)

folium.GeoJson(
    dummy_map, 
    name="Prince-Edward-County Boundary", 
    style_function=lambda feature: {"color": "black", "weight": 2, "fillOpacity": 0},
    tooltip=folium.GeoJsonTooltip(fields=["Boundary"])
).add_to(boundary_layer)

boundary_layer.add_to(m)

sampled_points = monte_carlo_sampling(dummy_map.geometry.iloc[0], num_points = 10)
m,point_layer,points_gdf= plot_sampled_points(sampled_points, dummy_map, m)
# m

In [8]:
df_prince_edward = pd.DataFrame(df_prince_edward)

# Create a geometry column from the Longitude and Latitude columns
df_prince_edward['geometry'] = df_prince_edward.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)

# Convert the DataFrame into a GeoDataFrame
df_prince_edward = gpd.GeoDataFrame(df_prince_edward, geometry='geometry', crs="EPSG:4326")
geometry_gdf = gpd.GeoDataFrame(df_prince_edward[['geometry']], geometry='geometry', crs="EPSG:4326")

geometry_gdf

,geometry
1260,POINT (-77.00704 44.07572)
1559,POINT (-77.58957 44.09635)
1560,POINT (-77.60212 44.09391)
1677,POINT (-77.57969 44.0482)
1907,POINT (-77.60006 44.06114)
2250,POINT (-77.14352 44.00616)
2400,POINT (-77.13588 44.02123)
2408,POINT (-77.16016 43.99984)
2423,POINT (-77.14679 44.00475)
2486,POINT (-77.13918 44.00934)


In [9]:
if dummy_map.crs.to_string() != "EPSG:4326":
    dummy_map = dummy_map.to_crs(epsg=4326)

In [10]:
geometry_gdf.crs.to_string()

'EPSG:4326'

In [11]:
polygon = dummy_map.geometry.iloc[0]
within_polygon = geometry_gdf.geometry.within(polygon)
geometry_gdf_within = geometry_gdf[within_polygon]

In [12]:
within_polygon_mall = gdf_mall.geometry.within(polygon)
mall_geometry_gdf_within = gdf_mall[within_polygon_mall]

In [13]:
# Filter parking polygons within the given polygon
within_polygon_parking = gdf_parking[gdf_parking.geometry.within(polygon)]

In [14]:
gas_layer = folium.FeatureGroup(name="Gas-Stations", 
                                     overlay=True, control=True)

for idx, row in geometry_gdf_within.iterrows():
    point = row.geometry
    folium.Marker(
        location=[point.y, point.x],  # (lat, lon)
        popup="Fuel"
    ).add_to(gas_layer)

gas_layer.add_to(m)


highway_layer = folium.FeatureGroup(name="Highway-Layer", 
                                     overlay=True, control=True)

geometry_highway = gpd.GeoDataFrame(gdf[['geometry']], geometry='geometry', crs="EPSG:4326")

for idx, row in geometry_highway.iterrows():
    folium.PolyLine(
        locations=[(lat, lon) for lon, lat in row['geometry'].coords],
        color="blue",  
        weight=2.5,    
        opacity=1.0).add_to(highway_layer)

highway_layer.add_to(m)



pop_layer = folium.FeatureGroup(name="Population-Layer",
                               overlay=True, control=True)

cities_to_add = ["Picton", "Wellington"]

for _, row in df_pop.iterrows():
    if row['City'] in cities_to_add:
        folium.Marker(
            location=[row['lat'], row['long']],
            popup=f"{row['City']}: Density {row['Population Density']}",
            icon=folium.Icon(color="red", icon="info-sign")
        ).add_to(pop_layer)

pop_layer.add_to(m)

mall_layer = folium.FeatureGroup(name="Mall-Stations", 
                                     overlay=True, control=True)

for idx, row in mall_geometry_gdf_within.iterrows():
    point = row.geometry
    folium.Marker(
        location=[point.y, point.x],  # (lat, lon)
        popup="Mall",
        icon=folium.Icon(color="green", icon="shopping-cart")
    ).add_to(mall_layer)

mall_layer.add_to(m)


# Create a new FeatureGroup for parking within the polygon
parking_layer = folium.FeatureGroup(name="Parking-Layer", overlay=True, control=True)

# Add each parking geometry to the FeatureGroup
for idx, row in within_polygon_parking.iterrows():
    if row.geometry.geom_type == 'Point':
        folium.Marker(
            location=[row.geometry.y, row.geometry.x],
            popup="Parking",
            icon=folium.Icon(color="pink", icon="info-sign")
        ).add_to(parking_layer)
    elif row.geometry.geom_type == 'Polygon':
        folium.GeoJson(
            row.geometry,
            style_function=lambda feature: {
                'fillColor': 'blue',
                'color': 'blue',
                'weight': 2,
                'fillOpacity': 0.5
            }
        ).add_to(parking_layer)

# Add the parking layer to the map
parking_layer.add_to(m)

# Ensure LayerControl is added for toggling layers
# folium.LayerControl(collapsed=False).add_to(m)

# # Display the map
# m

In [15]:
for key in m._children:
    print(key)

openstreetmap
feature_group_4be1dff8a830b8e1c8ab36dbfd19680e
feature_group_52fcd052a2f3817f503bc8f112d35234
feature_group_d34dffb8656eafd658c34b26d45841d6
feature_group_7c004194e12ef104583e6c04e22c5582
feature_group_da7ae606a1a6d56e4ae28d4da196444a
feature_group_2d9a59cef729c03e8f8a1ea1a598f6e0
feature_group_8f4f1b3f069a7ed437e2d82176011701


## Changes made specifically for PEI

In [16]:
filtered_gdf_pop = gdf_pop[gdf_pop['City'].isin(cities_to_add)][['City', 'Population', 'Population Density','Area Size','geometry']]
filtered_gdf_pop['Area'] = filtered_gdf_pop['Population'] / filtered_gdf_pop['Population Density']

In [17]:
updated_geometries = []

# Iterate over each geometry in within_polygon_parking
for geom in within_polygon_parking.geometry:
    if geom.geom_type == 'Polygon':
        # If the geometry is a polygon, extract its centroid
        updated_geometries.append(geom.centroid)
    elif geom.geom_type == 'Point':
        # If the geometry is already a point, include it as is
        updated_geometries.append(geom)

# Convert the list of updated geometries to a GeoSeries
updated_geometries = gpd.GeoSeries(updated_geometries)

In [18]:
moving_points = gpd.GeoDataFrame(
    data={
        "gas_station": geometry_gdf_within.geometry.reset_index(drop=True),
        "gen_points": points_gdf.geometry.reset_index(drop=True),
        "highway" : gpd.GeoSeries([Point(coord) for line in geometry_highway["geometry"] if line and line.geom_type == "LineString" for coord in line.coords],crs=geometry_highway.crs),
        "population": filtered_gdf_pop.geometry.reset_index(drop=True),
        "mall" : mall_geometry_gdf_within.geometry.reset_index(drop=True),
        "parking" : updated_geometries.reset_index(drop=True)

    } # This will reset the index automatically
)

moving_points.head(5)

,gas_station,gen_points,highway,population,mall,parking
0,POINT (-77.57969 44.0482),POINT (-77.0725 43.98346),POINT (-77.08499 44.13072),POINT (-77.1333 44),POINT (-77.14117 44.00694),POINT (-77.28996 44.01514)
1,POINT (-77.14352 44.00616),POINT (-77.01686 44.01658),POINT (-77.08597 44.1583),POINT (-77.3534 43.9579),None,POINT (-77.16143 43.99947)
2,POINT (-77.13588 44.02123),POINT (-77.41792 44.09487),POINT (-77.0863 44.16675),None,None,POINT (-77.23944 43.90647)
3,POINT (-77.16016 43.99984),POINT (-77.51668 43.95462),POINT (-77.08634 44.16735),None,None,POINT (-77.14145 44.00584)
4,POINT (-77.14679 44.00475),POINT (-77.21683 44.13915),POINT (-77.08645 44.17054),None,None,POINT (-77.14555 44.01312)


# Calculating Score and Moving

In [19]:
# Constant Variables
ACCEPTEBLE_DISTANCE = 10
FACTOR_STEP = 0.05
GAS_STATION_WEIGHT = 0.4
HIGHWAY_WEIGHT = 0.7
POPULATION_WEIGHT = 0.4
MALL_WEIGHT = 0.3
PARKING_WEIGHT = 0.5

FACTOR_NAME = ["gas_station", "highway", "population","mall", "parking"]

SCORE_GDF = copy.deepcopy(points_gdf)
SCORE_GDF = SCORE_GDF.rename(columns={'geometry': 'gen_points'})

population_score_gdf = copy.deepcopy(points_gdf)
gas_station_score_gdf = copy.deepcopy(points_gdf)
highway_score_gdf = copy.deepcopy(points_gdf)
mall_score_gdf = copy.deepcopy(points_gdf)
parking_score_gdf = copy.deepcopy(points_gdf)

parking_score_gdf = parking_score_gdf.rename(columns={'geometry': 'gen_points'})
mall_score_gdf = mall_score_gdf.rename(columns={'geometry': 'gen_points'})
gas_station_score_gdf = gas_station_score_gdf.rename(columns={'geometry': 'gen_points'})
population_score_gdf = population_score_gdf.rename(columns={'geometry': 'gen_points'})
highway_score_gdf = highway_score_gdf.rename(columns={'geometry': 'gen_points'})

highway_score_gdf["weight"] = HIGHWAY_WEIGHT
gas_station_score_gdf["weight"] = GAS_STATION_WEIGHT
population_score_gdf["weight"] = POPULATION_WEIGHT
mall_score_gdf["weight"] = MALL_WEIGHT
parking_score_gdf["weight"] = PARKING_WEIGHT

In [20]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c 


def calculate_bearing(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1

    x = np.sin(dlon) * np.cos(lat2)
    y = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(dlon)

    initial_bearing = np.degrees(np.arctan2(x, y))
    compass_bearing = (initial_bearing + 360) % 360  
    return compass_bearing 

In [21]:
def closest_factor_bearing(generated_points,factors):
    if generated_points is None:
        return None
    
    closest_distance = float("inf")
    nearest_station = None
    best_bearing = None
    
    for factor in factors:
        if factor is None:
            continue
        dist = haversine(generated_points.y, 
                         generated_points.x, 
                         factor.y,
                         factor.x)

        if dist < closest_distance:
            closest_distance = dist
            nearest_station = factor
            best_bearing = calculate_bearing(generated_points.y, 
                         generated_points.x, 
                         factor.y,
                         factor.x)
            
    if nearest_station is None:
        return None        
            
    return nearest_station, closest_distance, best_bearing

In [22]:
def factor_proximity_score(generated_points, factor_score_calculation_df, factor, factor_name, ACCEPTEBLE_DISTANCE=5, GLOBAL_SCORE_GDF=SCORE_GDF):
    """
    Parameters:
    generated_points : GeoSeries of shapely.geometry.Point
        Generated Points to be evaluated.
    factor_score_calculation_df : GeoDataFrame
        DataFrame for calculating the factor score.
    factor : GeoSeries of shapely.geometry.Point
        Points representing the factor (e.g., gas stations).
    factor_name : str
        Name of the factor (e.g., "gas_station").
    ACCEPTEBLE_DISTANCE : int, optional
        Acceptable distance within which the score is calculated. Default is 5.
    GLOBAL_SCORE_GDF : GeoDataFrame, optional
        Global DataFrame to store the scores. Default is SCORE_GDF.
    """
    factor_score_calculation_df[f"nearest_{factor_name}_point"], factor_score_calculation_df["distance"], factor_score_calculation_df[f"{factor_name}_bearing"] = zip(
        *generated_points.apply(lambda gp: closest_factor_bearing(gp, factor.tolist()))
    )
    GLOBAL_SCORE_GDF[f"{factor_name}_score"] = ACCEPTEBLE_DISTANCE / (factor_score_calculation_df["distance"] + ACCEPTEBLE_DISTANCE)
    factor_score_calculation_df[f"{factor_name}_score"] = GLOBAL_SCORE_GDF[f"{factor_name}_score"]
    return factor_score_calculation_df    

In [23]:
def calculate_step_distance(step, factor_weight, factor_score, distance):
    return step * factor_weight * factor_score * distance
# calculate_step_distance(FACTOR_STEP, GAS_STATION_WEIGHT, SCORE_GDF['gas_station_score'], gas_score_gdf['distance'])

In [24]:
def move_point(lat, lon, bearing_degrees,distance_step):
    R = 6371  
    bearing = np.radians(bearing_degrees)
    lat1, lon1 = np.radians(lat), np.radians(lon)
    
    lat2 = np.arcsin(np.sin(lat1) * np.cos(distance_step / R) +
                     np.cos(lat1) * np.sin(distance_step / R) * np.cos(bearing))
    
    lon2 = lon1 + np.arctan2(np.sin(bearing) * np.sin(distance_step / R) * np.cos(lat1),
                             np.cos(distance_step / R) - np.sin(lat1) * np.sin(lat2))
    point = Point(np.degrees(lon2), np.degrees(lat2))
    return point

### Calling Score Functions and moving individual Factors

In [25]:
factor_proximity_score(gas_station_score_gdf.gen_points, gas_station_score_gdf, moving_points.gas_station, "gas_station", ACCEPTEBLE_DISTANCE)
factor_proximity_score(highway_score_gdf.gen_points, highway_score_gdf, moving_points.highway, "highway", ACCEPTEBLE_DISTANCE = 5)
factor_proximity_score(population_score_gdf.gen_points, population_score_gdf, moving_points.population, "population", ACCEPTEBLE_DISTANCE)
factor_proximity_score(mall_score_gdf.gen_points, mall_score_gdf, moving_points.mall, "mall", ACCEPTEBLE_DISTANCE = 0.1)
factor_proximity_score(parking_score_gdf.gen_points, parking_score_gdf, moving_points.parking, "parking", ACCEPTEBLE_DISTANCE = 0.1)

for factor in FACTOR_NAME:
    factor_score_gdf = globals()[f"{factor}_score_gdf"]
    weight = globals()[f"{factor.upper()}_WEIGHT"]
    factor_score_gdf["distance_step"] = calculate_step_distance(FACTOR_STEP, weight, factor_score_gdf[f'{factor}_score'], factor_score_gdf['distance'])

In [26]:
gas_station_score_gdf['updated_points'] = gas_station_score_gdf.apply(
    lambda row: move_point(row.gen_points.y, row.gen_points.x, row.gas_station_bearing, row.distance_step),
    axis=1
)
gas_station_score_gdf.head()

,gen_points,weight,nearest_gas_station_point,distance,gas_station_bearing,gas_station_score,distance_step,updated_points
0,POINT (-77.0725 43.98346),0.4,POINT (-77.1391838 44.0093432),6.060944,298.370279,0.622628,0.075474,POINT (-77.07333 43.98378)
1,POINT (-77.01686 44.01658),0.4,POINT (-77.1358803 44.0212321),9.530907,273.149809,0.512009,0.097598,POINT (-77.01808 44.01663)
2,POINT (-77.41792 44.09487),0.4,POINT (-77.3160673 44.0675633),8.683965,110.427219,0.535218,0.092956,POINT (-77.41683 44.09457)
3,POINT (-77.51668 43.95462),0.4,POINT (-77.5796876 44.0481987),11.561871,334.181342,0.463782,0.107244,POINT (-77.51727 43.95549)
4,POINT (-77.21683 44.13915),0.4,POINT (-77.3160673 44.0675633),11.231286,224.904858,0.471003,0.105799,POINT (-77.21777 44.13847)


In [27]:
mall_score_gdf['updated_points'] = mall_score_gdf.apply(
    lambda row: move_point(row.gen_points.y, row.gen_points.x, row.mall_bearing, row.distance_step),
    axis=1
)
mall_score_gdf.head()

,gen_points,weight,nearest_mall_point,distance,mall_bearing,mall_score,distance_step,updated_points
0,POINT (-77.0725 43.98346),0.3,POINT (-77.1411661 44.00693884),6.081592,295.443132,0.016177,0.001476,POINT (-77.07252 43.98347)
1,POINT (-77.01686 44.01658),0.3,POINT (-77.1411661 44.00693884),9.998431,263.885328,0.009903,0.001485,POINT (-77.01688 44.01658)
2,POINT (-77.41792 44.09487),0.3,POINT (-77.1411661 44.00693884),24.182521,113.751059,0.004118,0.001494,POINT (-77.41791 44.09486)
3,POINT (-77.51668 43.95462),0.3,POINT (-77.1411661 44.00693884),30.604209,78.910931,0.003257,0.001495,POINT (-77.51666 43.95462)
4,POINT (-77.21683 44.13915),0.3,POINT (-77.1411661 44.00693884),15.895067,157.621813,0.006252,0.001491,POINT (-77.21682 44.13913)


In [28]:
parking_score_gdf['updated_points'] = parking_score_gdf.apply(
    lambda row: move_point(row.gen_points.y, row.gen_points.x, row.parking_bearing, row.distance_step),
    axis=1
)
parking_score_gdf.head()

,gen_points,weight,nearest_parking_point,distance,parking_bearing,parking_score,distance_step,updated_points
0,POINT (-77.0725 43.98346),0.5,POINT (-77.03395431660776 43.96694433764531),3.589891,120.759679,0.027101,0.002432,POINT (-77.07247 43.98345)
1,POINT (-77.01686 44.01658),0.5,POINT (-77.05631692909562 44.04000176429944),4.090295,309.553089,0.023865,0.002440,POINT (-77.01689 44.0166)
2,POINT (-77.41792 44.09487),0.5,POINT (-77.3924041 43.999301849999995),10.820184,169.125881,0.009157,0.002477,POINT (-77.41792 44.09484)
3,POINT (-77.51668 43.95462),0.5,POINT (-77.5252431 43.9581942),0.792305,300.137614,0.112069,0.002220,POINT (-77.51671 43.95463)
4,POINT (-77.21683 44.13915),0.5,POINT (-77.12353843300318 44.05516459959393),11.946011,141.385164,0.008302,0.002479,POINT (-77.21681 44.13913)


In [29]:
highway_score_gdf['updated_points'] = highway_score_gdf.apply(
    lambda row: move_point(row.gen_points.y, row.gen_points.x, row.highway_bearing, row.distance_step),
    axis=1
)
highway_score_gdf.head(5)

,gen_points,weight,nearest_highway_point,distance,highway_bearing,highway_score,distance_step,updated_points
0,POINT (-77.0725 43.98346),0.7,POINT (-77.1357289806699 44.018284431901264),6.369468,307.460115,0.439774,0.098039,POINT (-77.07347 43.984)
1,POINT (-77.01686 44.01658),0.7,POINT (-77.10063651945873 44.07275529518836),9.156742,313.038177,0.353189,0.113192,POINT (-77.0179 44.01728)
2,POINT (-77.41792 44.09487),0.7,POINT (-77.3964882944953 44.10128570039722),1.854601,67.353754,0.729437,0.047349,POINT (-77.41738 44.09503)
3,POINT (-77.51668 43.95462),0.7,POINT (-77.37386560390674 44.077962988105774),17.847631,39.733348,0.218841,0.136703,POINT (-77.51559 43.95556)
4,POINT (-77.21683 44.13915),0.7,POINT (-77.08499059560633 44.130716210718695),10.563215,95.045279,0.321270,0.118778,POINT (-77.21535 44.13905)


In [30]:
population_score_gdf['updated_points'] = population_score_gdf.apply(
    lambda row: move_point(row.gen_points.y, row.gen_points.x, row.population_bearing, row.distance_step),
    axis=1
)
population_score_gdf.head()

,gen_points,weight,nearest_population_point,distance,population_bearing,population_score,distance_step,updated_points
0,POINT (-77.0725 43.98346),0.4,POINT (-77.1333 44),5.199896,290.731258,0.657899,0.068420,POINT (-77.0733 43.98368)
1,POINT (-77.01686 44.01658),0.4,POINT (-77.1333 44),9.493059,258.839231,0.513003,0.097399,POINT (-77.01806 44.01641)
2,POINT (-77.41792 44.09487),0.4,POINT (-77.3534 43.9579),16.079867,161.265041,0.383438,0.123312,POINT (-77.41743 44.09382)
3,POINT (-77.51668 43.95462),0.4,POINT (-77.3534 43.9579),13.075202,88.343468,0.433366,0.113327,POINT (-77.51527 43.95465)
4,POINT (-77.21683 44.13915),0.4,POINT (-77.1333 44),16.850252,156.639283,0.372436,0.125513,POINT (-77.21621 44.13811)


### Adding All individual factor points to map

In [31]:
# Single Iteration Gas Points update
updated_gdf_gas = gpd.GeoDataFrame(gas_station_score_gdf[['updated_points']], geometry='updated_points', crs="EPSG:4326") 

updated_point_layer_gas = folium.FeatureGroup(name="Gas_Single_Itr_Point", 
                                     overlay=True, control=True, show = False)

for idx, row in updated_gdf_gas.iterrows():
    point = row.updated_points
    folium.CircleMarker(
        location=[point.y, point.x],  # (lat, lon)
        radius=3,
        color='green',
        fill = True,
        fill_color='green',
        fill_opacity=0.7,
        popup="Single-Itr Gas Updated point"
    ).add_to(updated_point_layer_gas)

updated_point_layer_gas.add_to(m)


# Single Iteration Highway Points update
updated_gdf_highway = gpd.GeoDataFrame(highway_score_gdf[['updated_points']], geometry='updated_points', crs="EPSG:4326") 

updated_point_layer_highway = folium.FeatureGroup(name="Highway_Single_Itr_Point", 
                                     overlay=True, control=True, show = False)

for idx, row in updated_gdf_highway.iterrows():
    point = row.updated_points
    folium.CircleMarker(
        location=[point.y, point.x],  # (lat, lon)
        radius=3,
        color='pink',
        fill = True,
        fill_color='pink',
        fill_opacity=0.7,
        popup="Singe-Itr Highway Updated point"
    ).add_to(updated_point_layer_highway)

updated_point_layer_highway.add_to(m)

# Single Iteration Population Points update
updated_gdf_population = gpd.GeoDataFrame(population_score_gdf[['updated_points']], geometry='updated_points', crs="EPSG:4326") 

updated_point_layer_population = folium.FeatureGroup(name="Population_Single_Itr_Point", 
                                     overlay=True, control=True, show = False)

for idx, row in updated_gdf_population.iterrows():
    point = row.updated_points
    folium.CircleMarker(
        location=[point.y, point.x],  # (lat, lon)
        radius=3,
        color='orange',
        fill = True,
        fill_color='orange',
        fill_opacity=0.7,
        popup="Singe-Itr Population Updated point"
    ).add_to(updated_point_layer_population)

updated_point_layer_population.add_to(m)

# Single Iteration Mall Points update
updated_gdf_mall = gpd.GeoDataFrame(mall_score_gdf[['updated_points']], geometry='updated_points', crs="EPSG:4326")

updated_point_layer_mall = folium.FeatureGroup(name="Mall_Single_Itr_Point",
                                     overlay=True, control=True, show = False)

for idx, row in updated_gdf_mall.iterrows():
    point = row.updated_points
    folium.CircleMarker(
        location=[point.y, point.x],  # (lat, lon)
        radius=3,
        color='purple',
        fill = True,
        fill_color='purple',
        fill_opacity=0.7,
        popup="Singe-Itr Mall Updated point"
    ).add_to(updated_point_layer_mall)

updated_point_layer_mall.add_to(m)

# Single Iteration Parking Points update
updated_gdf_parking = gpd.GeoDataFrame(parking_score_gdf[['updated_points']], geometry='updated_points', crs="EPSG:4326")

updated_point_layer_parking = folium.FeatureGroup(name="Parking_Single_Itr_Point",
                                     overlay=True, control=True, show = False)

for idx, row in updated_gdf_parking.iterrows():
    point = row.updated_points
    folium.CircleMarker(
        location=[point.y, point.x],  # (lat, lon)
        radius=3,
        color='blue',
        fill = True,
        fill_color='blue',
        fill_opacity=0.7,
        popup="Singe-Itr Parking Updated point"
    ).add_to(updated_point_layer_parking)

updated_point_layer_parking.add_to(m)


### Combining The Factors

Combining the Bearing:

east = magnitude * sin(bearing)

north = magnitude * cos(bearing)

In [32]:
import numpy as np
import pandas as pd

def combined_bearing_from_factor_dfs(factor_names):
    """
    Computes a combined bearing from multiple factor GeoDataFrames.
    
    For each factor in factor_names (e.g., "gas_station", "highway"),
    this function looks for a GeoDataFrame named "{factor}_score_gdf" that 
    has columns:
      - "{factor}_bearing"
      - "{factor}_score"
      - "weight"  (assumed to be present and valid for that factor)
      
    It then calculates for each row:
      net_east  = sum( weight * score * sin(radians(bearing)) )
      net_north = sum( weight * score * cos(radians(bearing)) )
      net_bearing = arctan2(net_east, net_north) converted to degrees (0-360)
      
    Assumes that each factor GeoDataFrame has the same row order (e.g. same gen_points).
    
    Parameters:
      factor_names (list): List of factor name strings (e.g., ["gas_station", "highway"])
      
    Returns:
      pd.Series: A Series of combined bearings (in degrees) for each row.
    """
    net_east = None
    net_north = None
    
    for factor in factor_names:
        # Get the factor GeoDataFrame from the global namespace.
        # For example, if factor is "gas_station", this fetches gas_station_score_gdf.
        df = globals()[f"{factor}_score_gdf"]
        
        # Get the factor-specific bearing and score columns.
        bearing_series = df[f"{factor}_bearing"]
        score_series   = df[f"{factor}_score"]
        weight_series  = df["weight"]  # Same column name in each dataframe.
        
        # Compute influence (you can adjust this formula if needed)
        influence = weight_series * score_series
        
        # Decompose the influence into east and north components.
        east_comp  = influence * np.sin(np.radians(bearing_series))
        north_comp = influence * np.cos(np.radians(bearing_series))
        
        if net_east is None:
            net_east = east_comp.copy()
            net_north = north_comp.copy()
        else:
            net_east += east_comp
            net_north += north_comp
    
    combined_bearing = (np.degrees(np.arctan2(net_east, net_north)) + 360) % 360
    return combined_bearing


In [33]:
SCORE_GDF["combined_bearing"] = combined_bearing_from_factor_dfs(FACTOR_NAME)
SCORE_GDF["combined_step"] = gas_station_score_gdf["distance_step"] + highway_score_gdf["distance_step"] + population_score_gdf["distance_step"] + mall_score_gdf["distance_step"] + parking_score_gdf["distance_step"]/ 5

In [34]:
SCORE_GDF['updated_points'] = SCORE_GDF.apply(
    lambda row: move_point(row.gen_points.y, row.gen_points.x, row.combined_bearing, row.combined_step),
    axis=1
)
SCORE_GDF.head(5)

,gen_points,gas_station_score,highway_score,population_score,mall_score,parking_score,combined_bearing,combined_step,updated_points
0,POINT (-77.0725 43.98346),0.622628,0.439774,0.657899,0.016177,0.027101,299.283822,0.243896,POINT (-77.07516 43.98454)
1,POINT (-77.01686 44.01658),0.512009,0.353189,0.513003,0.009903,0.023865,283.845186,0.310163,POINT (-77.02063 44.01725)
2,POINT (-77.41792 44.09487),0.535218,0.729437,0.383438,0.004118,0.009157,92.245826,0.265607,POINT (-77.4146 44.09477)
3,POINT (-77.51668 43.95462),0.463782,0.218841,0.433366,0.003257,0.112069,24.188491,0.359212,POINT (-77.51484 43.95756)
4,POINT (-77.21683 44.13915),0.471003,0.321270,0.372436,0.006252,0.008302,152.522993,0.352076,POINT (-77.2148 44.13634)


In [35]:
# Combined Factor Iteration
iterations = 100
for i in range(iterations):
    factor_proximity_score(SCORE_GDF.updated_points, highway_score_gdf, moving_points.highway, "highway", ACCEPTEBLE_DISTANCE = 5)
    factor_proximity_score(SCORE_GDF.updated_points, gas_station_score_gdf, moving_points.gas_station, "gas_station", ACCEPTEBLE_DISTANCE)
    factor_proximity_score(SCORE_GDF.updated_points, population_score_gdf, moving_points.population, "population", ACCEPTEBLE_DISTANCE)
    factor_proximity_score(SCORE_GDF.updated_points, mall_score_gdf, moving_points.mall, "mall", ACCEPTEBLE_DISTANCE = 0.1)
    factor_proximity_score(SCORE_GDF.updated_points, parking_score_gdf, moving_points.parking, "parking", ACCEPTEBLE_DISTANCE = 0.1)


    SCORE_GDF["combined_bearing"] = combined_bearing_from_factor_dfs(FACTOR_NAME)
    SCORE_GDF["combined_step"] = gas_station_score_gdf["distance_step"] + highway_score_gdf["distance_step"] + population_score_gdf["distance_step"] + mall_score_gdf["distance_step"] + parking_score_gdf["distance_step"]/ 5

    for factor in FACTOR_NAME:
        factor_score_gdf = globals()[f"{factor}_score_gdf"]
        weight = globals()[f"{factor.upper()}_WEIGHT"]
        factor_score_gdf["distance_step"] = calculate_step_distance(FACTOR_STEP, weight, factor_score_gdf[f'{factor}_score'], factor_score_gdf['distance'])

    SCORE_GDF['updated_points'] = SCORE_GDF.apply(
        lambda row: move_point(row.updated_points.y, row.updated_points.x, row.combined_bearing, row['combined_step']),
        axis=1
    )

### Final Candidate Score

In [36]:
SCORE_GDF['final_candidate_score'] = (((np.exp(GAS_STATION_WEIGHT) * SCORE_GDF['gas_station_score']) + (np.exp(HIGHWAY_WEIGHT) * SCORE_GDF['highway_score']) + (np.exp(POPULATION_WEIGHT) * SCORE_GDF['population_score']) + (np.exp(MALL_WEIGHT) * SCORE_GDF['mall_score']) + (np.exp(PARKING_WEIGHT) * SCORE_GDF['parking_score']))
                                      / (np.exp(GAS_STATION_WEIGHT) + np.exp(HIGHWAY_WEIGHT) + np.exp(POPULATION_WEIGHT) + np.exp(MALL_WEIGHT) + np.exp(PARKING_WEIGHT)))
SCORE_GDF

,gen_points,gas_station_score,highway_score,population_score,mall_score,parking_score,combined_bearing,combined_step,updated_points,final_candidate_score
0,POINT (-77.0725 43.98346),0.993510,0.836653,0.895734,0.210879,0.890149,125.352015,0.052377,POINT (-77.13806 44.00948),0.782330
1,POINT (-77.01686 44.01658),0.974324,0.997045,0.825957,0.066991,0.301352,78.172534,0.042690,POINT (-77.13531 44.01894),0.660429
2,POINT (-77.41792 44.09487),0.576139,0.976076,0.382966,0.004353,0.008900,76.922381,0.213182,POINT (-77.39421 44.09983),0.427332
3,POINT (-77.51668 43.95462),0.801268,0.984283,0.431743,0.005525,0.012240,346.927326,0.160131,POINT (-77.3452 44.07751),0.481389
4,POINT (-77.21683 44.13915),0.861176,0.989003,0.716252,0.030933,0.331314,95.869238,0.087644,POINT (-77.14062 44.03503),0.616916
5,POINT (-77.14069 43.88023),0.646400,0.981358,0.524964,0.011519,0.083459,194.375430,0.169827,POINT (-77.24573 43.98662),0.484848
6,POINT (-77.03854 44.02766),0.994250,0.993676,0.810104,0.057527,0.173497,229.060491,0.040907,POINT (-77.1356 44.0208),0.632380
7,POINT (-77.30447 44.05152),0.831839,0.998565,0.469146,0.007194,0.022879,106.146468,0.147979,POINT (-77.29989 44.05224),0.500143
8,POINT (-77.17293 44.07799),0.864848,0.986479,0.718476,0.031259,0.331186,3.110113,0.088136,POINT (-77.14087 44.0356),0.617410
9,POINT (-77.03735 44.00799),0.986467,0.848187,0.888652,0.184083,0.874352,298.421670,0.053735,POINT (-77.1394 44.01078),0.774819


### Only one factor iteration

In [37]:
# Only one factor iteration
# iterations = 10
# for i in range(iterations):
#     factor_proximity_score(gas_station_score_gdf.updated_points, gas_station_score_gdf, moving_points.gas_station, "gas_station", ACCEPTEBLE_DISTANCE)
#     gas_station_score_gdf['updated_points'] = gas_station_score_gdf.apply(
#         lambda row: move_point(row.updated_points.y, row.updated_points.x, row.gas_station_bearing, row['distance_step']),
#         axis=1
#     )

# Need careful thoughts on scoring formula 
1. Introduce a decay factor (Good for Road/highway -> Cause points are supposed to be extremely close to highway we could also play with acceptable distance) 
2. Inverse fuction approach (Good for Gas-station/Pop Density -> These factors do have some weights but its distance should not affect our overal score as much as highway) (In use)

Road/highway is the highest weighted factor:
NO ROAD = NO CHARGING STATION

Expression=α⋅wf⋅sf*(p) ⋅(∣∣pf−p∣∣)  -> (For step size how much to move its not a stander value rather a dynamic value that adjusts itself based on this formula)

Only move the point if the score is improving

sf*(p) = (ideal_proximity - current Proximity)

### TO DO:
1. Identify how may CS needed in a pop density city
    - Create a Circle around the city whose based of city's area. = city_proximity
    - Identify How many CS needed based on Pop/area ration i.e density = required_CS
    - Create penalty for all the points higher then required_CS inside the city_proximity. or ignore the city factor after required_CS with socre > 80% has been identified

2. Incorporate after solving the problem mathmatically. 

In [38]:
def add_updated_points_layer(m, dataframe, column_name , color="black", radius=3, popup_text="Updated_point"):
    # Dynamically create the column name from the factor_name

    # Create a GeoDataFrame using the dynamic column for the geometry
    updated_gdf = gpd.GeoDataFrame(dataframe[f"{column_name}"], geometry=column_name, crs="EPSG:4326")

    # Create a folium FeatureGroup for the updated points
    updated_point_layer__ = folium.FeatureGroup(name="Multi_Iteration_Updated_Points", overlay=True, control=True)

    # Add a CircleMarker for each point in the GeoDataFrame
    for idx, row in updated_gdf.iterrows():
        point = row[column_name]
        folium.CircleMarker(
            location=[point.y, point.x],  # Note: folium expects (lat, lon)
            radius=radius,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7,
            popup=dataframe['final_candidate_score'][idx]
        ).add_to(updated_point_layer__)

    # Add the feature group and layer control to the map
    updated_point_layer__.add_to(m)
    return m

add_updated_points_layer(m, SCORE_GDF, "updated_points", color="black", radius=3, popup_text="Multi-Iteration Updated Points")

folium.LayerControl(collapsed=False).add_to(m)
m

In [39]:
# updated_gdf_gas = gpd.GeoDataFrame(gas_station_score_gdf[['updated_points']], geometry='updated_points', crs="EPSG:4326") 

# updated_point_layer = folium.FeatureGroup(name="Updated_Points", 
#                                      overlay=True, control=True)

# for idx, row in updated_gdf_gas.iterrows():
#     point = row.updated_points
#     folium.CircleMarker(
#         location=[point.y, point.x],  # (lat, lon)
#         radius=3,
#         color='green',
#         fill = True,
#         fill_color='green',
#         fill_opacity=0.7,
#         popup="Updated_point"
#     ).add_to(updated_point_layer)

# updated_point_layer.add_to(m)
# folium.LayerControl(collapsed=False).add_to(m)

# m
